# Organize Json metadata to Excel format

**This notebook organizes the main fields from the metadata (scientific_integrity_dataset_v8.json) to a Excel format**


In [1]:
# Import Cell
import json
import pandas as pd
from glob import glob
import os, shutil

In [2]:
# Read the Json File
with open("scientific-integrity-dataset-v8/scientific_integrity_dataset_v8.json",'r') as f:
    sciintv8 = json.load(f)

In [88]:
# Create a Header to the Excel Spreadsheet
sciint_spreadsheet = pd.DataFrame(columns=[
                     'Directory',
                     'DOI',
                     'Title',
                     'Article Link',
                     'Authors',
                     'Cited By',
                     'Copyright',
                     'Publication Source',
                     'Published Date',
                     'Received Date',
                     'Accepted Date',
                     'Publisher',
                     'Has Retraction/Correction',
                     'Reason for Retreaction/Correction',
                     'Retraction/Correction Notice Available',
                     'Retraction/Correction DOI',
                     'Source Image Available',
                     'Supplmentary Material Available',
                     ]
)

### Fill the Spreadsheet with the metadata

In [89]:
index = 1
for key, item in sciintv8.items():
    new_entry = {}
    
    new_entry['Directory'] = key.replace('/',"_")
    if item['doi']:
        new_entry['DOI'] = item['doi']
    else:
        new_entry['DOI'] = '-'
    new_entry['Title'] = ' '.join(item['title'].replace('\n',' ').replace('\t',' ').split())
    new_entry['Article Link'] = item['article_url']
    
    authors = "-"
    if item['authors']:
        authors = ''
        for author in item['authors']:
            name = ""
            if 'given' in author.keys():
                name = author['given']
            if 'family' in author.keys():
                name = name +" " + author['family']
                name = name.strip()
            if authors:
                authors = authors+", "+name
            else:
                authors = name
        
    new_entry['Authors'] = authors
    new_entry['Cited By']  = item['cited_by']
    if new_entry['Cited By'] == None:
        new_entry['Cited By'] = '-'
    new_entry['Copyright'] = item['copyright']
    new_entry['Publication Source'] = item['publication_source']
    if item['article_history']['published_date']:
        new_entry['Published Date'] = item['article_history']['published_date'].replace('null','')
    else:
        new_entry['Published Date'] = '-'
    if item['article_history']['received_date']:
        new_entry['Received Date'] = item['article_history']['received_date']
    else:
        new_entry['Received Date'] = '-'
    if  item['article_history']['accepted_date']:       
        new_entry['Accepted Date'] = item['article_history']['accepted_date']
    else:
        new_entry['Accepted Date'] = ['-']
    new_entry['Publisher'] = item['publisher']
    
    
    
    if 'retraction_correction_material' in item.keys():
        new_entry['Has Retraction/Correction'] = 'Yes'
        if  item['retraction_correction_material']['retraction_correction_reason']:
            new_entry['Reason for Retreaction/Correction'] = item['retraction_correction_material']['retraction_correction_reason']
        else:
            new_entry['Reason for Retreaction/Correction'] = '-'
        if item['retraction_correction_material']['retraction_correction_notice_txt'] or item['retraction_correction_material']['retraction_correction_notice_pdf_link'] :
            new_entry['Retraction/Correction Notice Available'] = 'Yes'
        else:
            new_entry['Retraction/Correction Notice Available'] = 'No'
        if item['retraction_correction_material']['retraction_correction_doi']:
            new_entry['Retraction/Correction DOI'] = item['retraction_correction_material']['retraction_correction_doi']
        else:
             new_entry['Retraction/Correction DOI'] = '-'
        
    else:
        new_entry['Has Retraction/Correction'] = 'No'
        new_entry['Reason for Retreaction/Correction'] = '-'
        new_entry['Retraction/Correction Notice Available'] = '-'
        new_entry['Retraction/Correction DOI'] = '-'        
    
    if len(glob('scientific-integrity-dataset-v8/scientific-integrity-dataset-v8/'+key.replace('/',"_")+'/figures/*')) >= 1:
        new_entry['Source Image Available'] = 'Yes'
    else:
        new_entry['Source Image Available'] = 'No'
        
    if len(glob('scientific-integrity-dataset-v8/scientific-integrity-dataset-v8/'+key.replace('/',"_")+'/supplementary/*')) >= 1:
        new_entry['Supplmentary Material Available'] = 'Yes'
    else:
        new_entry['Supplmentary Material Available'] = 'Yes'
    sciint_spreadsheet = sciint_spreadsheet.append(pd.DataFrame(new_entry, index=[index]))
    index +=1
    

In [90]:
# Save to Excel SpreadSheet
sciint_spreadsheet.sort_values(by=['Publisher','Published Date']).reset_index().drop(columns=['index']).to_excel("data_annotation.xls")